In [1]:
%run ./label_data.ipynb

In [2]:
lst.head()

0    Toast will lay off 10% of its workforce, about...
1    About that AMC Networks class action lawsuit s...
2    DoorDash Beats Estimates But Shares Fall After...
3    Retail sales fall 0.8% in January from Decembe...
4    25+ Best Presidents Day Sales to Shop This Wee...
Name: description, dtype: object

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from tqdm import tqdm
import re

[nltk_data] Downloading package stopwords to C:\Users\Shahryar
[nltk_data]     Rza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Shahryar
[nltk_data]     Rza\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
def decontraction(phrase: str) -> str:
    map_list = [
        ["won't", "will not"],
        ["can't", "cannot"],
        ["n't", " not"],
        ["r'e", " are"],
        ["'s", " is"],
        ["'d", " would"],
        ["'ll", " will"],
        ["'t", " not"],
        ["'ve", " have"],
        ["'m", " am"],
        ["n't", " not"],
        ["'re", " are"],
        ["'s", " is"],
        
    ]
    
    for amap in map_list:
        phrase = re.sub(amap[0], amap[1], phrase)
    return phrase 



In [5]:
decontraction("Your aren't a good person. I'm a geek.")

'Your are not a good person. I am a geek.'

In [6]:
def preprocess(series_column: pd.Series, remove_stopword: bool) -> list:
    preprocessed = []
    characters_to_remove = ['\\r', '\\"', '\\n']
    for sentence in tqdm(series_column.values):
        sentence = str(sentence)
        for char in characters_to_remove:
            sentence = sentence.replace(char, ' ')
        sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
        sentence = decontraction(sentence)
        
        if remove_stopword:
            sentence = ' '.join(sent for sent in sentence.split() if sent not in stopwords.words())
        preprocessed.append(sentence.lower().strip())
    return preprocessed

In [7]:
lst['clean_text'] = preprocess(lst, True)

  0%|          | 0/1214 [00:00<?, ?it/s]

 57%|█████▋    | 689/1214 [08:10<06:02,  1.45it/s]  

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def generate_sentiment_scores(df: pd.DataFrame):
    def overall_sentiment(negative: float, positive: float) -> str:
        if negative > positive:
            return 'negative'
        elif positive > negative:
            return 'positive'
        else:
            return 'neutral'
    sid = SentimentIntensityAnalyzer()
    neg, pos, overall = [], [], []
    for sentence in tqdm(df['clean_text']):
        sentence_sentiment_score = sid.polarity_scores(sentence)
        neg.append(sentence_sentiment_score['neg'])
        pos.append(sentence_sentiment_score['pos'])
        overall.append(overall_sentiment(sentence_sentiment_score['neg'], sentence_sentiment_score['pos']))
    return neg, pos, overall
        

In [ ]:
lst['negative'], lst['positive'], lst['overall'] = generate_sentiment_scores(lst)

  0%|          | 0/1214 [00:00<?, ?it/s]

100%|██████████| 1214/1214 [00:00<00:00, 5513.05it/s]


In [ ]:
dfd = pd.DataFrame(lst)
dfd['negative'].transpose()
lst.to_csv('labeled_sentiment.csv')


KeyError: 'negative'

In [ ]:
# lst_pos = [0.136, 0.0, 0.276, 0.122, 0.229, 0.227, 0.0, 0.08, 0.065, 0.0, 0.138, 0.054, 0.076, 0.0, 0.055, 0.0, 0.0, 0.145, 0.125, 0.118, 0.067, 0.191, 0.115, 0.083, 0.208, 0.08, 0.0, 0.123, 0.13, 0.102, 0.075, 0.176, 0.117, 0.105, 0.078, 0.098, 0.0, 0.212, 0.104, 0.0, 0.041, 0.19, 0.0, 0.065, 0.046, 0.103, 0.0, 0.067, 0.038, 0.0, 0.0, 0.0, 0.21, 0.11, 0.272, 0.0, 0.0, 0.0, 0.111, 0.113, 0.167, 0.0, 0.187, 0.017, 0.031, 0.188, 0.079, 0.06, 0.168, 0.0, 0.0, 0.032, 0.03, 0.061, 0.204, 0.0, 0.078, 0.0, 0.203, 0.016, 0.0, 0.0, 0.158, 0.235, 0.07, 0.0, 0.319, 0.09, 0.0, 0.144, 0.169, 0.0, 0.208, 0.0, 0.0, 0.26, 0.234, 0.426, 0.073, 0.142, 0.0, 0.143, 0.041, 0.067, 0.148, 0.178, 0.106, 0.106, 0.069, 0.0, 0.049, 0.0, 0.205, 0.103, 0.074, 0.0, 0.039, 0.095, 0.092, 0.042, 0.056, 0.088, 0.0, 0.041, 0.164, 0.122, 0.0, 0.072, 0.13, 0.0, 0.066, 0.092, 0.045, 0.164, 0.166, 0.27, 0.251, 0.0, 0.035, 0.438, 0.269, 0.156, 0.0, 0.197, 0.239, 0.108, 0.286, 0.471, 0.357, 0.056, 0.118, 0.398, 0.103, 0.203, 0.076, 0.2, 0.202, 0.1, 0.0, 0.0, 0.189, 0.196, 0.26, 0.248, 0.446, 0.163, 0.228, 0.108, 0.0, 0.231, 0.123, 0.484, 0.142, 0.312, 0.161, 0.225, 0.173, 0.0, 0.257, 0.169, 0.248, 0.053, 0.419, 0.043, 0.0, 0.106, 0.14, 0.0, 0.16, 0.075, 0.156, 0.161, 0.189, 0.172, 0.176, 0.0, 0.158, 0.088, 0.231, 0.186, 0.217, 0.057, 0.088, 0.236, 0.144, 0.0, 0.262, 0.145, 0.197, 0.201, 0.178, 0.213, 0.251, 0.39, 0.233, 0.134, 0.103, 0.156, 0.113, 0.153, 0.198, 0.078, 0.086, 0.0, 0.238, 0.157, 0.0, 0.0, 0.335, 0.085, 0.23, 0.079, 0.081, 0.377, 0.368, 0.0, 0.39, 0.0, 0.233, 0.134, 0.049, 0.118, 0.249, 0.465, 0.108, 0.141, 0.287, 0.399, 0.339, 0.185, 0.304, 0.222, 0.104, 0.182, 0.157, 0.19, 0.088, 0.249, 0.333, 0.062, 0.221, 0.159, 0.324, 0.263, 0.202, 0.171, 0.333, 0.325, 0.057, 0.139, 0.355, 0.288, 0.191, 0.199, 0.144, 0.248, 0.355, 0.376, 0.262, 0.174, 0.0, 0.277, 0.245, 0.269, 0.352, 0.232, 0.254, 0.202, 0.056, 0.211, 0.0, 0.265, 0.36, 0.0, 0.25, 0.287, 0.222, 0.104, 0.182, 0.157, 0.19, 0.088, 0.249, 0.333, 0.062, 0.221, 0.159, 0.324, 0.263, 0.202, 0.171, 0.333, 0.325, 0.057, 0.139, 0.355, 0.288, 0.191, 0.199, 0.144, 0.248, 0.355, 0.376, 0.262, 0.174, 0.0, 0.277, 0.245, 0.269, 0.352, 0.232, 0.254, 0.202, 0.056, 0.211, 0.0, 0.265, 0.36, 0.0, 0.25, 0.287, 0.142, 0.22, 0.152, 0.142, 0.145, 0.178, 0.294, 0.06, 0.16, 0.151, 0.096, 0.147, 0.216, 0.231, 0.215, 0.068, 0.136, 0.109, 0.154, 0.193, 0.0, 0.102, 0.173, 0.031, 0.209, 0.173, 0.071, 0.12, 0.205, 0.236, 0.062, 0.281, 0.106, 0.28, 0.206, 0.155, 0.073, 0.062, 0.28, 0.066, 0.346, 0.0, 0.179, 0.158, 0.163, 0.174, 0.127, 0.263, 0.113, 0.207, 0.292, 0.321, 0.163, 0.16, 0.066, 0.154, 0.425, 0.094, 0.205, 0.115, 0.206, 0.223, 0.04, 0.162, 0.215, 0.13, 0.128, 0.177, 0.172, 0.122, 0.224, 0.04, 0.082, 0.359, 0.116, 0.049, 0.103, 0.162, 0.083, 0.065, 0.099, 0.114, 0.067, 0.263, 0.2, 0.13, 0.2, 0.138, 0.097, 0.196, 0.098, 0.108, 0.246, 0.194, 0.196, 0.122, 0.058, 0.117, 0.335, 0.187, 0.142, 0.202, 0.161, 0.172, 0.167, 0.351, 0.172, 0.257, 0.154, 0.255, 0.149, 0.5, 0.312, 0.141, 0.348, 0.0, 0.33, 0.423, 0.391, 0.171, 0.191, 0.0, 0.343, 0.085, 0.317, 0.266, 0.234, 0.0, 0.057, 0.16, 0.26, 0.228, 0.206, 0.401, 0.2, 0.245, 0.198, 0.249, 0.125, 0.0, 0.293, 0.261, 0.343, 0.085, 0.198, 0.231, 0.316, 0.0, 0.403, 0.252, 0.41, 0.091, 0.197, 0.183, 0.5, 0.067, 0.195, 0.209, 0.377, 0.381, 0.189, 0.26, 0.379, 0.418, 0.184, 0.344, 0.0, 0.25, 0.335, 0.213, 0.167, 0.191, 0.191, 0.174, 0.318, 0.257, 0.135, 0.312, 0.0, 0.13, 0.273, 0.307, 0.139, 0.29, 0.49, 0.247, 0.093, 0.191, 0.255, 0.263, 0.302, 0.349, 0.394, 0.062, 0.215, 0.283, 0.321, 0.0, 0.198, 0.0, 0.0, 0.282, 0.195, 0.205, 0.107, 0.0, 0.0, 0.129, 0.338, 0.0, 0.106, 0.203, 0.288, 0.167, 0.286, 0.325, 0.079, 0.069, 0.345, 0.218, 0.195, 0.267, 0.0, 0.0, 0.273, 0.363, 0.062, 0.246, 0.233, 0.428, 0.0, 0.144, 0.0, 0.309, 0.0, 0.191, 0.12, 0.353, 0.322, 0.116, 0.108, 0.067, 0.0, 0.101, 0.255, 0.0, 0.0, 0.0, 0.0, 0.0, 0.263, 0.0, 0.0, 0.0, 0.318, 0.0, 0.16, 0.197, 0.206, 0.297, 0.255, 0.0, 0.224, 0.0, 0.0, 0.0, 0.0, 0.161, 0.24, 0.076, 0.131, 0.306, 0.273, 0.183, 0.141, 0.046, 0.202, 0.162, 0.191, 0.061, 0.353, 0.174, 0.151, 0.0, 0.179, 0.373, 0.288, 0.0, 0.097, 0.482, 0.132, 0.173, 0.403, 0.294, 0.117, 0.0, 0.296, 0.062, 0.197, 0.18, 0.314, 0.159, 0.39, 0.257, 0.346, 0.154, 0.176, 0.109, 0.268, 0.213, 0.207, 0.251, 0.397, 0.235, 0.085, 0.0, 0.0, 0.144, 0.167, 0.305, 0.378, 0.055, 0.323, 0.335, 0.102, 0.079, 0.216, 0.091, 0.173, 0.0, 0.078, 0.092, 0.172, 0.197, 0.333, 0.137, 0.094, 0.346, 0.164, 0.27, 0.08, 0.125, 0.156, 0.082, 0.39, 0.161, 0.164, 0.087, 0.08, 0.347, 0.219, 0.0, 0.349, 0.302, 0.073, 0.0, 0.269, 0.297, 0.277, 0.08, 0.146, 0.113, 0.22, 0.218, 0.245, 0.341, 0.333, 0.306, 0.069, 0.16, 0.133, 0.0, 0.0, 0.158, 0.187, 0.248, 0.0, 0.094, 0.156, 0.231, 0.332, 0.085, 0.328, 0.0, 0.208, 0.241, 0.0, 0.248, 0.189, 0.154, 0.117, 0.155, 0.302, 0.394, 0.214, 0.0, 0.189, 0.191, 0.056, 0.251, 0.278, 0.239, 0.08, 0.17, 0.14, 0.203, 0.467, 0.0, 0.259, 0.201, 0.359, 0.269, 0.368, 0.091, 0.146, 0.0, 0.0, 0.412, 0.399, 0.0, 0.22, 0.104, 0.457, 0.088, 0.249, 0.167, 0.32, 0.325, 0.0, 0.162, 0.218, 0.0, 0.133, 0.118, 0.128, 0.111, 0.0, 0.0, 0.0, 0.091, 0.0, 0.141, 0.231, 0.267, 0.096, 0.0, 0.466, 0.0, 0.0, 0.0, 0.0, 0.144, 0.368, 0.0, 0.162, 0.0, 0.0, 0.268, 0.342, 0.0, 0.157, 0.128, 0.457, 0.13, 0.0, 0.0, 0.132, 0.197, 0.0, 0.353, 0.155, 0.0, 0.188, 0.0, 0.0, 0.345, 0.0, 0.0, 0.097, 0.0, 0.0, 0.237, 0.299, 0.0, 0.155, 0.129, 0.0, 0.123, 0.0, 0.203, 0.148, 0.292, 0.0, 0.261, 0.0, 0.0, 0.121, 0.172, 0.0, 0.355, 0.243, 0.107, 0.133, 0.421, 0.0, 0.0, 0.257, 0.276, 0.246, 0.215, 0.08, 0.0, 0.368, 0.0, 0.329, 0.283, 0.124, 0.095, 0.197, 0.236, 0.148, 0.155, 0.18, 0.304, 0.226, 0.103, 0.0, 0.137, 0.524, 0.079, 0.0, 0.228, 0.281, 0.235, 0.0, 0.0, 0.155, 0.13, 0.0, 0.415, 0.385, 0.0, 0.271, 0.328, 0.366, 0.149, 0.267, 0.325, 0.0, 0.414, 0.211, 0.278, 0.097, 0.333, 0.089, 0.233, 0.123, 0.167, 0.189, 0.274, 0.22, 0.309, 0.1, 0.277, 0.257, 0.211, 0.187, 0.305, 0.478, 0.0, 0.351, 0.145, 0.136, 0.339, 0.253, 0.361, 0.231, 0.391, 0.399, 0.144, 0.291, 0.279, 0.283, 0.281, 0.17, 0.156, 0.22, 0.277, 0.331, 0.278, 0.472, 0.435, 0.417, 0.227, 0.1, 0.242, 0.11, 0.298, 0.114, 0.286, 0.302, 0.181, 0.259, 0.258, 0.375, 0.412, 0.14, 0.179, 0.0, 0.316, 0.165, 0.204, 0.0, 0.261, 0.279, 0.101, 0.094, 0.317, 0.18, 0.0, 0.0, 0.234, 0.209, 0.22, 0.106, 0.135, 0.104, 0.075, 0.08, 0.304, 0.0, 0.109, 0.115, 0.076, 0.0, 0.132, 0.266, 0.087, 0.173, 0.157, 0.17, 0.384, 0.196, 0.286, 0.116, 0.0, 0.194, 0.317, 0.199, 0.126, 0.187, 0.265, 0.287, 0.195, 0.0, 0.135, 0.169, 0.083, 0.249, 0.183, 0.169, 0.11, 0.136, 0.289, 0.193, 0.371, 0.0, 0.278, 0.351, 0.097, 0.17, 0.066, 0.19, 0.08, 0.095, 0.138, 0.183, 0.115, 0.315, 0.219, 0.0, 0.0, 0.119, 0.212, 0.142, 0.273, 0.246, 0.0, 0.0, 0.196, 0.175, 0.107, 0.085, 0.283, 0.217, 0.211, 0.196, 0.0, 0.303, 0.0, 0.105, 0.0, 0.048, 0.062, 0.381, 0.259, 0.169, 0.205, 0.305, 0.201, 0.24, 0.173, 0.169, 0.301, 0.175, 0.312, 0.178, 0.168, 0.238, 0.103, 0.057, 0.086, 0.352, 0.335, 0.221, 0.161, 0.135, 0.179, 0.052, 0.336, 0.224, 0.31, 0.169, 0.235, 0.509, 0.223, 0.298, 0.143, 0.332, 0.427, 0.347, 0.234, 0.266, 0.157, 0.103, 0.178, 0.043, 0.291, 0.1, 0.264, 0.136, 0.231, 0.149, 0.272, 0.249, 0.318, 0.157, 0.176, 0.151, 0.138, 0.273, 0.135, 0.05, 0.225, 0.263, 0.234, 0.058, 0.211, 0.237, 0.378, 0.141, 0.164, 0.216, 0.0, 0.055, 0.188, 0.192, 0.17, 0.198, 0.288, 0.105, 0.059, 0.189, 0.077, 0.138, 0.141, 0.161, 0.274, 0.151, 0.056, 0.15, 0.117, 0.26, 0.165, 0.0, 0.213, 0.276, 0.194, 0.08, 0.22, 0.144, 0.194, 0.188, 0.145, 0.16, 0.331, 0.0, 0.399, 0.268, 0.163, 0.275, 0.0, 0.2, 0.167, 0.442, 0.0, 0.0, 0.0, 0.0, 0.113, 0.27, 0.231, 0.139, 0.108, 0.263, 0.114, 0.273, 0.274, 0.15, 0.0, 0.091, 0.129, 0.143, 0.146, 0.089, 0.0, 0.124, 0.0, 0.244, 0.113, 0.146, 0.125, 0.106, 0.0, 0.289, 0.222, 0.359, 0.424, 0.0, 0.0, 0.157, 0.259, 0.151, 0.0, 0.34, 0.0, 0.162, 0.14, 0.067, 0.0, 0.215, 0.0, 0.126, 0.073, 0.111, 0.069, 0.268, 0.167, 0.109, 0.11, 0.22, 0.106, 0.132, 0.21]

In [ ]:
# df_pos = pd.DataFrame(lst_pos)
# df_pos.to_csv('positive.csv')